## What is Ollama?
https://ollama.com/bibhup_mishra 

Ollama is a platform where you can run open source models like liama 2, phi 3, Gemma and others.

It provides CLI to run the models in local machine also.

In [1]:
# Step1 : download te ollama cli 
curl -fsSL https://ollama.com/install.sh | sh

# Step2: Run the model localy (default port: 127.0.0.1:11434)
ollama run gemma:2b

# Step3: Test the model with a simple curl command
curl http://localhost:11434/api/chat -d '{
  "model": "gemma:2b",
  "messages": [
    { "role": "user", "content": "why is the sky blue?" }
  ]
}'

Step3: 
curl http://localhost:11434/api/generate -d '{
  "model": "gemma:2b",
  "prompt":"Why is the sky blue?"
}'

SyntaxError: invalid decimal literal (1083917087.py, line 5)

In [ ]:
# install ollama package
! pip install langchain-ollama

In [2]:
## Step 1: Import the OllamaEmbeddings class and create an instance with the desired model
from utility.llm_factory import LLMFactory

embeddings = LLMFactory.get_embedding_model("ollama", model="gemma:2b")


r1= embeddings.embed_query("Why is the sky blue?")
r2= embeddings.embed_documents(["The sky is blue because of the way sunlight interacts with the Earth's atmosphere."])
print(r1)
print(r2)


/home/azureuser/miniconda3/envs/.langchain/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[0.00076872157, -0.019213416, -0.008689137, 0.03020918, -0.0011058724, 0.021015989, 0.0119799785, 0.008338877, -0.004229785, 0.0025991732, 0.032369774, -0.016321886, -0.0023087005, 0.013908445, -0.011950198, -0.0032995811, 0.05690532, 0.005916174, 0.022359116, 0.0062118215, 0.013209054, 0.007717311, -0.022377018, -0.022063585, 0.009836868, -0.009285204, 0.0019502953, -0.0067608426, 0.007194619, -0.028740201, 0.0049748155, -0.011860698, 0.011742001, -0.0074565825, -0.012014053, 0.011836124, 0.013480809, -0.008654571, -0.004449441, -0.019115053, -8.179612e-05, 0.010507498, -0.004765659, -0.014397191, -0.012774139, 0.017359897, 0.0017811512, -0.0038390737, -0.011238587, -0.0174357, -0.21755473, -0.25963914, -0.024701022, 0.007949947, -0.0016346914, -0.0166021, -0.018737251, 0.01715368, -0.031420603, -0.029692557, 0.00064671075, 0.009986916, -0.006761062, -0.005136711, -0.01648295, -0.00886116, -0.017778244, -0.01571438, -0.029571269, 0.02650832, 0.005752192, 0.013466888, -0.02104608, 0.00

In [ ]:
## Step2: Load a text file as a document
from langchain_community.document_loaders import TextLoader

loader = TextLoader('./_data/speech.txt', encoding='utf-8')
docs = loader.load()

docs

[Document(metadata={'source': './_data/speech.txt'}, page_content="Good morning everyone,\n\nToday, I want to talk about something incredibly simple, yet profoundly powerful: small steps.\n\nIn a world obsessed with big wins and overnight success, we often forget that every great achievement starts with a single small action.\n\nWhether you're trying to learn a new skill, change a habit, or build something meaningful — it always begins with the decision to take one small step forward.\n\nThink about the tallest buildings. They're built one brick at a time. Olympic athletes? They train for years, often making tiny improvements day after day.\n\nSo, the next time you feel overwhelmed by your goals, just focus on the next step. Not the next ten, not the whole staircase — just the next one.\n\nProgress isn’t always loud. Sometimes, it whispers.\n\nBut those whispers? They build momentum.\n\nAnd that momentum? It builds success.\n\nThank you.\n\n")]

In [ ]:
## Step 3: Split the document into smaller chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter


with open('./_data/speech.txt', 'r', encoding='utf-8') as f:
    speech = f.read()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=80,
    chunk_overlap=20
)

#final_documents = text_splitter.split_text(speech)

# Note: the input here is a list of text strings, not Document objects.
document_chunks = text_splitter.create_documents([speech])
document_chunks[::]  # Display the first two chunks

[Document(metadata={}, page_content='Good morning everyone,'),
 Document(metadata={}, page_content='Today, I want to talk about something incredibly simple, yet profoundly'),
 Document(metadata={}, page_content='yet profoundly powerful: small steps.'),
 Document(metadata={}, page_content='In a world obsessed with big wins and overnight success, we often forget that'),
 Document(metadata={}, page_content='often forget that every great achievement starts with a single small action.'),
 Document(metadata={}, page_content="Whether you're trying to learn a new skill, change a habit, or build something"),
 Document(metadata={}, page_content='or build something meaningful — it always begins with the decision to take one'),
 Document(metadata={}, page_content='to take one small step forward.'),
 Document(metadata={}, page_content="Think about the tallest buildings. They're built one brick at a time. Olympic"),
 Document(metadata={}, page_content='at a time. Olympic athletes? They train for yea

In [ ]:
## Step4: Create embeddings for the document chunks
## and store them in a vector database
from langchain.vectorstores import Chroma
vector_store_db = Chroma.from_documents(
    document_chunks,
    embeddings,
    collection_name="speech_collection"
)
# Check the number of documents in the vector store
print(f"Number of documents in the vector store: {vector_store_db._collection.count()}")

Number of documents in the vector store: 17


In [ ]:
## Query the vector store
query = "Progress isn’t always loud. Sometimes, it whispers."
results = vector_store_db.similarity_search(query, k=3)
print(type(results))
# Display the results
for i, result in enumerate(results):
    print(f"Result {i+1}:")
    print(result.page_content)
    print("-" * 40)

docs_and_scores = vector_store_db.similarity_search_with_score(query, k=3)
docs_and_scores[::]

<class 'list'>
Result 1:
Progress isn’t always loud. Sometimes, it whispers.
----------------------------------------
Result 2:
But those whispers? They build momentum.
----------------------------------------
Result 3:
often making tiny improvements day after day.
----------------------------------------


[(Document(metadata={}, page_content='Progress isn’t always loud. Sometimes, it whispers.'),
  0.0),
 (Document(metadata={}, page_content='But those whispers? They build momentum.'),
  0.3776080310344696),
 (Document(metadata={}, page_content='often making tiny improvements day after day.'),
  0.42664042115211487)]